<a href="https://colab.research.google.com/github/vriadi/CS614-Gen-AI-with-LLMs/blob/main/CS614_Individual_Assignment_mC4(id).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

Guided From Unsloth

https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Gemma3_(4B).ipynb#scrollTo=1pY6QSfT7kG_

# Installation

In [ ]:
%%capture
import os, re
import torch

if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

In [ ]:
from unsloth import FastModel

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit"
]

BASE_MODEL = "unsloth/gemma-3-4b-it"

model, tokenizer = FastModel.from_pretrained(
    model_name = BASE_MODEL,
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.3: Fast Gemma3 patching. Transformers: 4.55.4.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

# Current base model performance

In [ ]:
# BASE_SYSTEM = "You are a helpful Indonesian travel phrase assistant."
# STYLE_HINT = (
#     "When the user asks how to say something, reply with:\n"
#     "- Most common & natural: **<phrase>** (pronunciation: <...>) — short explanation.\n"
#     "- Slightly more formal: **<phrase>** (pronunciation: <...>) — short explanation.\n"
#     "- Optional polite variant if relevant.\n"
#     "Keep it concise and do not repeat the prompt."
# )

In [ ]:
BASE_SYSTEM = "You are a helpful Indonesian travel phrase assistant. Your task is to translate English to Indonesian."
STYLE_HINT = (
    "When the user asks how to say something, reply with:\n"
    "**<phrase>** (pronunciation: <...>) — short explanation.\n"
    "Keep it concise and do not repeat the prompt."
)

In [ ]:
# BASE_SYSTEM = "You are a helpful Indonesian language translator assistant."
# STYLE_HINT = (
#     "When the user asks a phrase, reply with:\n **<phrase>** (pronunciation phonetic: <...>)"
#     "Keep it concise and do not repeat the prompt."
# )

In [ ]:
def _get_text_tokenizer(tok):
    # If tok is a Processor, it will have .tokenizer
    return getattr(tok, "tokenizer", tok)

def chat_generate_clean(
    model, tokenizer, prompt,
    system=f"{BASE_SYSTEM}\n\n{STYLE_HINT}",
    max_new_tokens=128, do_sample=False, temperature=None, top_p=None
):
    model.eval()
    tok = _get_text_tokenizer(tokenizer)          # ← use real tokenizer
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    tok.padding_side = "right"

    msgs = [
        {"role": "system", "content": system},
        {"role": "user",   "content": prompt},
    ]

    device = getattr(model, "device", torch.device("cuda" if torch.cuda.is_available() else "cpu"))

    apply_ct = getattr(tok, "apply_chat_template", None)
    if callable(apply_ct):
        templated = tok.apply_chat_template(       # ← return a string, not tensors
            msgs, add_generation_prompt=True, tokenize=False
        )
        enc = tok(templated, return_tensors="pt", padding=True, truncation=True)
    else:
        # Fallback manual tags
        text = f"<|system|>\n{system}\n<|user|>\n{prompt}\n<|assistant|>\n"
        enc = tok(text, return_tensors="pt", padding=True, truncation=True)

    enc = {k: v.to(device) for k, v in enc.items()}
    prompt_len = enc["input_ids"].shape[1]

    with torch.no_grad():
        out = model.generate(
            **enc,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            temperature=temperature if do_sample else None,
            top_p=top_p if do_sample else None,
            pad_token_id=tok.eos_token_id,
            eos_token_id=tok.eos_token_id,
            return_dict_in_generate=True,
        )

    new_tokens = out.sequences[0, prompt_len:]     # decode only the completion
    return tok.decode(new_tokens, skip_special_tokens=True).strip()

In [ ]:
# prompts = [
#     "I want to order coffee.",
#     "Bill please",
#     "Where is the train station?",
#     "How much is the bus fare to the airport?",
#     "translate What is the most popular dish?"
# ]

prompts = [
    "I want to order coffee.",
    "Can I get the bill please?",
    "Where is the train station?",
    "How much is the bus fare to the airport?",
    "What is the most popular dish?"
]

In [ ]:
baseline = [chat_generate_clean(model, tokenizer, p) for p in prompts]

In [ ]:
import pandas as pd
pd.DataFrame({"Prompt": prompts, "Baseline": baseline})

,Prompt,Baseline
0,I want to order coffee.,**Saya mau kopi** (pronounced: sah-yah mahu ko...
1,Can I get the bill please?,"**""Minta tolong faktur""** (pronounced: min-tah..."
2,Where is the train station?,Stasiun kereta api di mana? (pronounced: stasi...
3,How much is the bus fare to the airport?,**Berapa harga tiket bus ke bandara?** (pronou...
4,What is the most popular dish?,Nasi goreng **(nah-see go-reong)** — Fried ric...


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


# Add LoRA adapters so we only need to update a small amount of parameters!

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model.language_model` require gradients


# Dataset
We will train the model in bahasa Indonesia using these datasets:
- mC4 Indonesian https://huggingface.co/datasets/allenai/c4 <br>
Under allenai/c4 with per-language configs. Indonesian is "id".

In [ ]:
from datasets import load_dataset, Dataset
from itertools import islice

# --- Ensure tokenizer has a pad token ---
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# --- Materialize a small clean slice from the stream ---
mc4_stream = load_dataset("allenai/c4", "id", split="train", streaming=True)
N = 10000  # Keep small for Colab
mc4_small = Dataset.from_list([{"text": ex["text"]} for ex in islice(mc4_stream, N)])

README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Note:
- The C4 dataset is HUGE (hundreds of gigabytes) — you cannot fully load it into Colab RAM. Even the id dataset has 69.5M rows

- Using .streaming=True allows lazy loading (reads line by line, not loading whole dataset).

- But streamed datasets cannot be shuffled or batched efficiently.

- So, we take only the first N samples and convert them into a normal HuggingFace Dataset with Dataset.from_list(...).

- This makes it trainable, shufflable, compatible with DataLoader, and stable for SFT.

In [ ]:
# --- Force every row to be a single trimmed string; drop empty/None ---
def _clean(r):
    txt = r.get("text")
    if txt is None:
        return {"text": ""}
    # Some samples can be lists/bytes/etc; coerce to string
    if isinstance(txt, (list, tuple)):
        txt = " ".join(map(str, txt))
    else:
        txt = str(txt)
    return {"text": txt.strip()}

mc4_small = mc4_small.map(_clean, num_proc=1)

# filter out empties / ultra-short lines (optional but helps)
mc4_small = mc4_small.filter(lambda r: len(r["text"]) > 1)

# keep ONLY the 'text' column so nothing else is inferred
mc4_small = mc4_small.remove_columns([c for c in mc4_small.column_names if c != "text"])

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Cleans the text:

- Removes None, blank, extra whitespace.

- Joins list-type text into a single string.

- Converts everything to clean, trimmed UTF-8 text.

- Removes garbage rows (empty / 1-character lines like "." or "-").

- Keeps only the text column → prevents HuggingFace from trying to process unknown fields.

In [ ]:
mc4_small

Dataset({
    features: ['text'],
    num_rows: 10000
})

In [ ]:
# for i in range(3):
#     print(f"[{i}] {mc4_small[i]['text']}\n")

mc4_small[0,2,3]

{'text': ['thapki full serial | Cinta Sinopsis\nHome » Search Results for "thapki full serial" Query\nthapki full serial\nDibawah ini telah kami tampilkan 10 sinopsis yang berkaitan dengan kata kunci sinopsis yang anda cari yaitu thapki full serial',
  'Cara Instalasi Cepat Network Camera Plug n View IC-3115W ~ All about IOT Home\n4. Antena Fixed - Antena fixed nirkabel. Harap antena ditegak untuk penerimaan sinyal yang baik. Bagian Belakang\n1. Konektor Tripod - Lubang mounting ini kompatibel dengan sebagian besar tripod kamera atau kamera berdiri , sehingga anda dapat menempatkan kamera di tempat yang aman.\n2. Port LAN - Untuk dihubungkan ke jaringan lokal area.\n3. Jack Power - Untuk Hubungkan ke adaptor 5V DC.\n4. Tombol WPS / Reset - Tekan tombol ini selama 3-5 detik untuk mulai koneksi WPS. Jika kamera jaringan tidak berfungsi dengan baik, anda dapat menekan tombol reset untuk me-reset kamera IP . Anda juga dapat menekan dan menahan tombol ini selama lebih dari 10 detik untuk me

Large language models (LLMs) like Gemma-3 are causal language models. They are trained to predict the next token in a sequence.

So, a single text (like a sentence in Indonesian) can act as both input and target:

Input: "Saya ingin memesan "

Target: "kopi."

The model sees the text, predicts the next token at each step, and is trained to minimize cross-entropy loss against the actual tokens.

This is essentially next-token prediction, which is supervised at the token level — every token is a label for the previous context.

## SFT

From Huggingface: https://huggingface.co/docs/trl/en/sft_trainer#expected-dataset-type-and-format

TRL supports the Supervised Fine-Tuning (SFT) Trainer for training language models. SFT supports both language modeling and prompt-completion datasets. The SFTTrainer is compatible with both standard and conversational dataset formats. When provided with a conversational dataset, the trainer will automatically apply the chat template to the dataset.


https://huggingface.co/docs/trl/en/sft_trainer#trl.trainer.sft_trainer.DataCollatorForLanguageModel…

Data collator used for language modeling data. Inputs are dynamically padded to the maximum length of a batch.

In [ ]:
from trl import SFTTrainer, SFTConfig
from transformers import DataCollatorForLanguageModeling

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
# mlm=False → not masked language modeling (like BERT), but causal LM (predict next token).
# The collator dynamically pads sequences and prepares input-target pairs for next-token prediction.

args = SFTConfig(
    dataset_text_field="text",
    # per_device_train_batch_size=2,
    per_device_train_batch_size=4,
    # gradient_accumulation_steps=4,
    gradient_accumulation_steps=8,
    warmup_steps=100,
    # max_steps=30,
    max_steps=300,
    learning_rate=1e-4, # lower LR often stabilizes LoRA fine-tuning
    logging_steps=50,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    report_to="none",
    padding_free=False,   # classic path
    packing=False,        # classic path
    max_seq_length=1024,
    remove_unused_columns=True,   # ← IMPORTANT
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=mc4_small,      # a regular HF Dataset with only a "text" column
    args=args,
    data_collator=collator,
)

trainer.train()
trainer.save_model("outputs_mc4/adapter")


Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/10000 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 1 | Total steps = 300
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 14,901,248 of 4,314,980,720 (0.35% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
50,2.979500
100,2.605800
150,2.557500
200,2.511800
250,2.481500
300,2.424000


# Test fine-tuned model performance

In [ ]:
gemma_finetuned = [chat_generate_clean(model, tokenizer, p) for p in prompts]

In [ ]:
import pandas as pd
pd.DataFrame({"Prompt": prompts, "Baseline": baseline, "Finetuned": gemma_finetuned})

,Prompt,Baseline,Finetuned
0,I want to order coffee.,**Saya mau kopi** (pronounced: sah-yah mahu ko...,**Saya mau pesan kopi.** (pronunciation: Sa-ya...
1,Can I get the bill please?,"**""Minta tolong faktur""** (pronounced: min-tah...","**Tagihannya, tolong.** (tagih-an-nya, to-long..."
2,Where is the train station?,Stasiun kereta api di mana? (pronounced: stasi...,Stasiun kereta api di mana? (Pronunciation: st...
3,How much is the bus fare to the airport?,**Berapa harga tiket bus ke bandara?** (pronou...,Harga bus ke bandara berapa? (pronounced: ha-r...
4,What is the most popular dish?,Nasi goreng **(nah-see go-reong)** — Fried ric...,Nasi goreng **(na-see go-reong)** — Fried rice...


# cendol-llama2-7b-inst

https://huggingface.co/indonlp/cendol-llama2-7b-inst

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

# pipe = pipeline("text-generation", model="indonlp/cendol-llama2-7b-inst")

pipe = pipeline(
    "text-generation",
    model="indonlp/cendol-llama2-7b-inst",
    device_map="auto",
    load_in_8bit=True   # saves VRAM
)


config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("indonlp/cendol-llama2-7b-inst")
model = AutoModelForCausalLM.from_pretrained("indonlp/cendol-llama2-7b-inst")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Simple translation function
def translate(text, max_new_tokens=64):
    # prompt = f"Tolong terjemahkan kalimat Inggris berikut ke dalam bahasa Indonesia: \n{text}\n Indonesian:"
    prompt = f"Terjemahkan kalimat bahasa Inggris ke bahasa Indonesia dan sertakan cara pengucapan fonetiknya: \n{text}\n Indonesian:"
    # prompt = f"Terjemahkan bahasa Inggris berikut ini ke bahasa Indonesia dan berikan juga cara pengucapan fonetiknya: \n{text}\n Indonesian:"
#     prompt = f"""
# Kamu adalah tutor bahasa untuk traveler. Tugasmu:
# 1. Terjemahkan kalimat bahasa Inggris ke bahasa Indonesia.
# 2. Gunakan gaya bicara yang alami dan sopan, bukan terlalu formal seperti buku teks.
# 3. Sertakan cara pengucapan fonetik dalam tanda kurung.
# 4. Format jawaban dengan jelas seperti contoh English:'Bill please' Indonesian: 'Bill, tolong' (bil toh-long)
# Tidak perlu mengulang prompt.
# Sekarang terjemahkan:
# English: "{text}"
# Indonesian:
# """
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            do_sample=False
        )

    # decode only the generated part
    generated = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return generated.strip()


In [ ]:
cendol = [translate(p) for p in prompts]

In [ ]:
# import pandas as pd
pd.DataFrame({"Prompt": prompts, "Cendol": cendol})

,Prompt,Cendol
0,I want to order coffee.,Saya ingin memesan kopi.
1,Can I get the bill please?,Dapatkah saya mendapatkan tagihannya?
2,Where is the train station?,Dimana stasiun kereta apinya?
3,How much is the bus fare to the airport?,Berapa ongkos bus ke bandara?
4,What is the most popular dish?,Apa makanan yang paling populer?


# Evaluate

## Human evaluation

In [ ]:
pd.DataFrame({"Prompt": prompts, "Base": baseline, "Finetuned": gemma_finetuned, "Cendol": cendol})

,Prompt,Base,Finetuned,Cendol
0,I want to order coffee.,**Saya mau kopi** (pronounced: sah-yah mahu ko...,**Saya mau pesan kopi.** (pronunciation: Sa-ya...,Saya ingin memesan kopi.
1,Can I get the bill please?,"**""Minta tolong faktur""** (pronounced: min-tah...","**Tagihannya, tolong.** (tagih-an-nya, to-long...",Dapatkah saya mendapatkan tagihannya?
2,Where is the train station?,Stasiun kereta api di mana? (pronounced: stasi...,Stasiun kereta api di mana? (Pronunciation: st...,Dimana stasiun kereta apinya?
3,How much is the bus fare to the airport?,**Berapa harga tiket bus ke bandara?** (pronou...,Harga bus ke bandara berapa? (pronounced: ha-r...,Berapa ongkos bus ke bandara?
4,What is the most popular dish?,Nasi goreng **(nah-see go-reong)** — Fried ric...,Nasi goreng **(na-see go-reong)** — Fried rice...,Apa makanan yang paling populer?


## Quantitatively against Cendol
Use Cendol’s outputs as a reference to see how close the finetuned Gemma model is.

### BLEU and ROUGE Metrics

| **BLEU (Bilingual Evaluation Understudy)** | **ROUGE (Recall-Oriented Understudy for Gisting Evaluation)** |
| :----------------------------------------- | :------------------------------------------------------------ |
| Measures n-gram overlap between generated text and reference text. | Focuses on recall of overlapping n-grams. |
| Commonly used for machine translation. | More often used in summarization but can also evaluate translations. |
| Scores range from 0 → 1 (or 0 → 100 if scaled). Higher is better. | Several variants: <br> • **ROUGE-N**: n-gram overlap (like BLEU but recall-focused) <br> • **ROUGE-L**: Longest common subsequence (captures fluency and structure) <br> Scores range 0 → 1. |
| **Limitation:** Does not consider meaning, grammar, or synonymy—only exact matches. | **Limitation:** Similar—still based on n-gram overlap, though recall-oriented. |
| Emphasizes **precision** (how much of generated text matches reference). | Emphasizes **recall** (how much of reference is covered by generated text). |


In [ ]:
!pip install nltk rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=73d2f715f0dcdd2f69d5c27a473afdf86a00b332e7028152a445079fc14ebc5a
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

# Initialize smoothing for BLEU
smooth_fn = SmoothingFunction().method1

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

bleu_scores = []
rouge1_scores = []
rougeL_scores = []

for gemma, cendol in zip(gemma_finetuned, cendol):
    # BLEU score (single reference)
    bleu = sentence_bleu([cendol.split()], gemma.split(), smoothing_function=smooth_fn)
    bleu_scores.append(bleu)

    # ROUGE score
    rouge = scorer.score(cendol, gemma)
    rouge1_scores.append(rouge['rouge1'].fmeasure)
    rougeL_scores.append(rouge['rougeL'].fmeasure)

In [ ]:
first_metrics = pd.DataFrame({
    "Prompt": prompts,
    "Base translation": baseline,
    "Finetuned translation": gemma_finetuned,
    "Cendol": cendol,
    "BLEU": bleu_scores,
    "ROUGE-1": rouge1_scores,
    "ROUGE-L": rougeL_scores,
})

first_metrics

,Prompt,Base translation,Finetuned translation,Cendol,BLEU,ROUGE-1,ROUGE-L
0,I want to order coffee.,**Saya mau kopi** (pronounced: sah-yah mahu ko...,**Saya mau pesan kopi.** (pronunciation: Sa-ya...,Apa makanan yang paling populer?,0.000000,0.142857,0.142857
1,Can I get the bill please?,"**""Minta tolong faktur""** (pronounced: min-tah...","**Tagihannya, tolong.** (tagih-an-nya, to-long...",Apa makanan yang paling populer?,0.000000,0.133333,0.133333
2,Where is the train station?,Stasiun kereta api di mana? (pronounced: stasi...,Stasiun kereta api di mana? (Pronunciation: st...,Apa makanan yang paling populer?,0.024325,0.200000,0.200000
3,How much is the bus fare to the airport?,**Berapa harga tiket bus ke bandara?** (pronou...,Harga bus ke bandara berapa? (pronounced: ha-r...,Apa makanan yang paling populer?,0.017515,0.242424,0.181818
4,What is the most popular dish?,Nasi goreng **(nah-see go-reong)** — Fried ric...,Nasi goreng **(na-see go-reong)** — Fried rice...,Apa makanan yang paling populer?,0.000000,0.000000,0.000000


## chrF (Character F-score)
https://machinetranslate.org/chrF

chrF (CHaRacter-level F-score) is a metric for machine translation evaluation that calculates the similarity between a machine translation output and a reference translation using character n-grams, not word n-grams.



In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.2 MB/s eta 0:00:00


In [ ]:
from sacrebleu.metrics import CHRF

base_chrF = []
chrf = CHRF()

for pred, ref in zip(baseline, cendol):
    # normalize
    pred_norm = pred.lower().strip()
    ref_norm = ref.lower().strip()

    bchrf_score = chrf.sentence_score(pred_norm, [ref_norm]).score
    base_chrF.append(bchrf_score)

chrF_scores = []
chrf = CHRF()

for pred, ref in zip(gemma_finetuned, cendol):
    # normalize
    pred_norm = pred.lower().strip()
    ref_norm = ref.lower().strip()

    chrf_score = chrf.sentence_score(pred_norm, [ref_norm]).score
    chrF_scores.append(chrf_score)

print(f"Average chrF2: {sum(chrF_scores)/len(chrF_scores):.2f}")

Average chrF2: 2.08


In [ ]:
second_metric = pd.DataFrame({
    "Prompt": prompts,
    "Base translation": baseline,
    "Finetuned translation": gemma_finetuned,
    "Cendol": cendol,
    "Baseline chrF2": base_chrF,
    "Finetuned chrF2": chrF_scores
})

second_metric

,Prompt,Base translation,Finetuned translation,Cendol,Baseline chrF2,Finetuned chrF2
0,I want to order coffee.,**Saya mau kopi** (pronounced: sah-yah mahu ko...,**Saya mau pesan kopi.** (pronunciation: Sa-ya...,Apa makanan yang paling populer?,4.587156,4.854369
1,Can I get the bill please?,"**""Minta tolong faktur""** (pronounced: min-tah...","**Tagihannya, tolong.** (tagih-an-nya, to-long...",Apa makanan yang paling populer?,3.875969,0.000000
2,Where is the train station?,Stasiun kereta api di mana? (pronounced: stasi...,Stasiun kereta api di mana? (Pronunciation: st...,Apa makanan yang paling populer?,4.629630,5.555556
3,How much is the bus fare to the airport?,**Berapa harga tiket bus ke bandara?** (pronou...,Harga bus ke bandara berapa? (pronounced: ha-r...,Apa makanan yang paling populer?,0.000000,0.000000
4,What is the most popular dish?,Nasi goreng **(nah-see go-reong)** — Fried ric...,Nasi goreng **(na-see go-reong)** — Fried rice...,Apa makanan yang paling populer?,6.097561,0.000000


### All quants against Cendol

In [ ]:
df_metrics = pd.DataFrame({
    "Prompt": prompts,
    "BLEU": bleu_scores,
    "ROUGE-1": rouge1_scores,
    "ROUGE-L": rougeL_scores,
    "chrF": chrF_scores
})

df_metrics

,Prompt,BLEU,ROUGE-1,ROUGE-L,chrF
0,I want to order coffee.,0.000000,0.142857,0.142857,4.854369
1,Can I get the bill please?,0.000000,0.133333,0.133333,0.000000
2,Where is the train station?,0.024325,0.200000,0.200000,5.555556
3,How much is the bus fare to the airport?,0.017515,0.242424,0.181818,0.000000
4,What is the most popular dish?,0.000000,0.000000,0.000000,0.000000


## Quantitatively reference text
Use reference text from Indonesian native speaker as a reference to see how close the finetuned Gemma model is. This is only humanely possible if there's small sample.

In [ ]:
from sacrebleu.metrics import CHRF

# Multiple references per sentence (you can start with one each)
references = [
    ["Saya mau kopi", "Saya ingin pesan kopi"],
    ["Boleh minta bon?", "Minta tagihan ya?"],
    ["Stasiun kereta api dimana?", "Dimana stasiun kereta api?"],
    ["Berapa harga tiket bus ke bandara?", "Berapa ongkos bus ke bandara?"],
    ["Apa makanan yang paling populer?", "Makanan apa yang paling populer?"]
]

import re

def normalize(s):
    s = s.lower().strip()
    s = re.sub(r"[.,?!]", "", s)
    return s

smooth_fn = SmoothingFunction().method1
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
chrf = CHRF()

bleu_scores = []
rouge1_scores = []
rougeL_scores = []
chrF_scores = []

# gemma_finetuned = list of model outputs (one per sentence)
for pred, refs in zip(gemma_finetuned, references):
    pred_norm = normalize(pred)
    refs_norm = [normalize(r) for r in refs]

    # BLEU
    bleu = sentence_bleu([r.split() for r in refs_norm], pred_norm.split(), smoothing_function=smooth_fn)
    bleu_scores.append(bleu)

    # ROUGE
    rouge = scorer.score(" ".join(pred_norm.split()), " ".join(refs_norm[0].split()))
    rouge1_scores.append(rouge['rouge1'].fmeasure)
    rougeL_scores.append(rouge['rougeL'].fmeasure)

    # chrF
    chrf_score = chrf.sentence_score(pred_norm, refs_norm).score
    chrF_scores.append(chrf_score)


print(f"Average BLEU: {sum(bleu_scores)/len(bleu_scores):.2f}")
print(f"Average ROUGE-1: {sum(rouge1_scores)/len(rouge1_scores):.2f}")
print(f"Average ROUGE-L: {sum(rougeL_scores)/len(rougeL_scores):.2f}")
print(f"Average chrF2: {sum(chrF_scores)/len(chrF_scores):.2f}")

# Optional: display per-sentence table
import pandas as pd

df = pd.DataFrame({
    "Prompt": prompts,
    "Finetuned": gemma_finetuned,
    "BLEU": bleu_scores,
    "ROUGE-1": rouge1_scores,
    "ROUGE-L": rougeL_scores,
    "chrF": chrF_scores
})

df


Average BLEU: 0.02
Average ROUGE-1: 0.16
Average ROUGE-L: 0.15
Average chrF2: 34.53


,Prompt,Finetuned,BLEU,ROUGE-1,ROUGE-L,chrF
0,I want to order coffee.,**Saya mau pesan kopi.** (pronunciation: Sa-ya...,0.010331,0.222222,0.222222,27.638665
1,Can I get the bill please?,"**Tagihannya, tolong.** (tagih-an-nya, to-long...",0.000000,0.000000,0.000000,30.701905
2,Where is the train station?,Stasiun kereta api di mana? (Pronunciation: st...,0.056930,0.300000,0.300000,61.470143
3,How much is the bus fare to the airport?,Harga bus ke bandara berapa? (pronounced: ha-r...,0.046575,0.294118,0.235294,40.963749
4,What is the most popular dish?,Nasi goreng **(na-see go-reong)** — Fried rice...,0.000000,0.000000,0.000000,11.868953
